In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm

from collections import Counter
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder

# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', 20)
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

# Загружаем информацию о клиентах и их продуктах

In [2]:
customer_products = pd.read_csv('cust_prod_v2.csv')
customer_products

,CUSTOMER_ID,PRODUCT_NAME=0,PRODUCT_NAME=1,PRODUCT_NAME=2,PRODUCT_NAME=3,PRODUCT_NAME=4,PRODUCT_NAME=5,PRODUCT_NAME=6,PRODUCT_NAME=7,PRODUCT_NAME=8,PRODUCT_NAME=9,PRODUCT_NAME=10,PRODUCT_NAME=11,PRODUCT_NAME=12,PRODUCT_NAME=13,PRODUCT_NAME=14,PRODUCT_NAME=15,PRODUCT_NAME=16,PRODUCT_NAME=17,PRODUCT_NAME=18,PRODUCT_NAME=19,PRODUCT_NAME=20,PRODUCT_NAME=21,PRODUCT_NAME=22,PRODUCT_NAME=23,PRODUCT_NAME=24,PRODUCT_NAME=25,PRODUCT_NAME=26,PRODUCT_NAME=27,PRODUCT_NAME=28,PRODUCT_NAME=29,PRODUCT_NAME=30,PRODUCT_NAME=31,PRODUCT_NAME=32,PRODUCT_NAME=33,PRODUCT_NAME=34,PRODUCT_NAME=35,PRODUCT_NAME=36,PRODUCT_NAME=37,PRODUCT_NAME=38,PRODUCT_NAME=39,PRODUCT_NAME=40,PRODUCT_NAME=41,PRODUCT_NAME=42,PRODUCT_NAME=43,PRODUCT_NAME=44,PRODUCT_NAME=45,PRODUCT_NAME=46,PRODUCT_NAME=47,PRODUCT_NAME=48,PRODUCT_NAME=49,PRODUCT_NAME=50,PRODUCT_NAME=51,PRODUCT_NAME=52,PRODUCT_NAME=53,PRODUCT_NAME=54,PRODUCT_NAME=55,PRODUCT_NAME=56,PRODUCT_NAME=57,PRODUCT_NAME=58,PRODUCT_NAME=59,PRODUCT_NAME=60,PRODUCT_NAME=61,PRODUCT_NAME=62,PRODUCT_NAME=63,PRODUCT_NAME=64,PRODUCT_NAME=65,PRODUCT_NAME=66,PRODUCT_NAME=67,PRODUCT_NAME=68,PRODUCT_NAME=69,PRODUCT_NAME=70,PRODUCT_NAME=71,PRODUCT_NAME=72,PRODUCT_NAME=73,PRODUCT_NAME=74,PRODUCT_NAME=75,PRODUCT_NAME=76,PRODUCT_NAME=77,PRODUCT_NAME=78,PRODUCT_NAME=79,PRODUCT_NAME=80,PRODUCT_NAME=81,PRODUCT_NAME=82,PRODUCT_NAME=83,PRODUCT_NAME=84,PRODUCT_NAME=85,PRODUCT_NAME=86,PRODUCT_NAME=87,PRODUCT_NAME=88,PRODUCT_NAME=89,PRODUCT_NAME=90,PRODUCT_NAME=91,PRODUCT_NAME=92,PRODUCT_NAME=93,PRODUCT_NAME=94,PRODUCT_NAME=95,PRODUCT_NAME=96,PRODUCT_NAME=97,PRODUCT_NAME=98,PRODUCT_NAME=99,PRODUCT_NAME=100,PRODUCT_NAME=101,PRODUCT_NAME=102,PRODUCT_NAME=103,PRODUCT_NAME=104,PRODUCT_NAME=105,PRODUCT_NAME=106,PRODUCT_NAME=107,PRODUCT_NAME=108,PRODUCT_NAME=109,PRODUCT_NAME=110,PRODUCT_NAME=111,PRODUCT_NAME=112,PRODUCT_NAME=113,PRODUCT_NAME=114,PRODUCT_NAME=115,PRODUCT_NAME=116,PRODUCT_NAME=117,PRODUCT_NAME=118,PRODUCT_NAME=119,PRODUCT_NAME=120,PRODUCT_NAME=121,PRODUCT_NAME=122,PRODUCT_NAME=123,PRODUCT_NAME=124,PRODUCT_NAME=125,PRODUCT_NAME=126,PRODUCT_NAME=127,PRODUCT_NAME=128,PRODUCT_NAME=129,PRODUCT_NAME=130,PRODUCT_NAME=131,PRODUCT_NAME=132,PRODUCT_NAME=133,PRODUCT_NAME=134,PRODUCT_NAME=135,PRODUCT_NAME=136,PRODUCT_NAME=137,PRODUCT_NAME=138,PRODUCT_NAME=139,PRODUCT_NAME=140,PRODUCT_NAME=141,PRODUCT_NAME=142,PRODUCT_NAME=143,PRODUCT_NAME=144,PRODUCT_NAME=145,PRODUCT_NAME=146,PRODUCT_NAME=147,PRODUCT_NAME=148,PRODUCT_NAME=149,PRODUCT_NAME=150,PRODUCT_NAME=151,PRODUCT_NAME=152,PRODUCT_NAME=153,PRODUCT_NAME=154,PRODUCT_NAME=155,PRODUCT_NAME=156,PRODUCT_NAME=157,PRODUCT_NAME=158,PRODUCT_NAME=159,PRODUCT_NAME=160,PRODUCT_NAME=161,PRODUCT_NAME=162,PRODUCT_NAME=163,PRODUCT_NAME=164,PRODUCT_NAME=165,PRODUCT_NAME=166,PRODUCT_NAME=167,PRODUCT_NAME=168,PRODUCT_NAME=169,PRODUCT_NAME=170,PRODUCT_NAME=171,PRODUCT_NAME=172,PRODUCT_NAME=173,PRODUCT_NAME=174,PRODUCT_NAME=175,PRODUCT_NAME=176,PRODUCT_NAME=177,PRODUCT_NAME=178,PRODUCT_NAME=179,PRODUCT_NAME=180,PRODUCT_NAME=181,PRODUCT_NAME=182,PRODUCT_NAME=183,PRODUCT_NAME=184,PRODUCT_NAME=185,PRODUCT_NAME=186,PRODUCT_NAME=187,PRODUCT_NAME=188,PRODUCT_NAME=189,PRODUCT_NAME=190,PRODUCT_NAME=191,PRODUCT_NAME=192,PRODUCT_NAME=193,PRODUCT_NAME=194,PRODUCT_NAME=195,PRODUCT_NAME=196,PRODUCT_NAME=197,PRODUCT_NAME=198,PRODUCT_NAME=199,PRODUCT_NAME=200,PRODUCT_NAME=201,PRODUCT_NAME=202,PRODUCT_NAME=203,PRODUCT_NAME=204,PRODUCT_NAME=205,PRODUCT_NAME=206,PRODUCT_NAME=207,PRODUCT_NAME=208,PRODUCT_NAME=209,PRODUCT_NAME=210,PRODUCT_NAME=211,PRODUCT_NAME=212,PRODUCT_NAME=213,PRODUCT_NAME=214,PRODUCT_NAME=215,PRODUCT_NAME=216,PRODUCT_NAME=217,PRODUCT_NAME=218,PRODUCT_NAME=219,PRODUCT_NAME=220,PRODUCT_NAME=221,PRODUCT_NAME=222,PRODUCT_NAME=223,PRODUCT_NAME=224,PRODUCT_NAME=225,PRODUCT_NAME=226,PRODUCT_NAME=227,PRODUCT_NAME=228,PRODUCT_NAME=229,PRODUCT_NAME=230,PRODUCT_NAME=231,PRODUCT_NAME=232,PRODUCT_NAME=233,PRODUCT_NAME=234,PRODUCT_NAME=235,PRODUCT_NAME=236,PRODUCT_NAME=237,PRODUCT_NAME=238,PRODUCT_NAME=239,PRODUCT_NAME=240,

In [3]:
train_df = pd.read_csv('train.csv')
train_df

,TENURE,CITY,is_churn,BILLS_NUM,INV_MON_1,INV_MON_2,INV_MON_3,INV_MON_4,INV_MON_5,INV_MON_6,N_LATE_FEES,N_NON_SETTLED,AVG_BILL,MIN_BILL,MAX_BILL,MAX2MIN_BILL,PAY_NUM,AVG_PAY,MAX_PAY,MIN_PAY,PAY_MON_1,PAY_MON_2,PAY_MON_3,PAY_MON_4,PAY_MON_5,PAY_MON_6,MAX2MIN_PAY,ADJ_NUM,AVG_ADJ,ADJ_MON_1,ADJ_MON_2,ADJ_MON_3,ADJ_MON_4,ADJ_MON_5,ADJ_MON_6,MIN_ADJ,MAX_ADJ,MAXMINDIFF_ADJ,DSON_NUM_BUNDLE,DSON_MON_1_BUNDLE,DSON_MON_2_BUNDLE,DSON_MON_3_BUNDLE,DSON_MON_4_BUNDLE,DSON_MON_5_BUNDLE,DSON_MON_6_BUNDLE,DSON_NUM_PROMO,DSON_MON_1_PROMO,DSON_MON_2_PROMO,DSON_MON_3_PROMO,DSON_MON_4_PROMO,DSON_MON_5_PROMO,DSON_MON_6_PROMO,DSOF_NUM_BUNDLE,DSOF_MON_1_BUNDLE,DSOF_MON_2_BUNDLE,DSOF_MON_3_BUNDLE,DSOF_MON_4_BUNDLE,DSOF_MON_5_BUNDLE,DSOF_MON_6_BUNDLE,DSOF_NUM_PROMO,DSOF_MON_1_PROMO,DSOF_MON_2_PROMO,DSOF_MON_3_PROMO,DSOF_MON_4_PROMO,DSOF_MON_5_PROMO,DSOF_MON_6_PROMO,INTERACTS_NUM,TFT,SUPPORT,SOCIAL_MEDIA,OUTBOUND_CALL,MANAGE_ACCT,INQUIRY,INBOUND_CALL,ESCALATION,EMAIL,CALL_BACK,CTI,PLAY_TV,PLAY_PHONE,PLAY_INET,SINGLE_PLAY,DOUBLE_PLAY,TRIPLE_PLAY,UNKNOWN_PLAY,INV_MON_A,INV_MON_B,PAY_MON_A,PAY_MON_B,ADJ_MON_A,ADJ_MON_B,DSON_MONPROMO_A,DSON_MONPROMO_B,DSOF_MONPROMO_A,DSOF_MONPROMO_B,DSON_MONBUNDLE_A,DSON_MONBUNDLE_B,DSOF_MONBUNDLE_A,DSOF_MONBUNDLE_B,ITR_CAT_119,ITR_CAT_120,ITR_CAT_114,ITR_CAT_9,ITR_CAT_123,ITR_CAT_128,ITR_CAT_102,ITR_CAT_104,ITR_CAT_115,ITR_CAT_8,ITR_CAT_122,ITR_CAT_0,ITR_CAT_22,ITR_CAT_133,ITR_CAT_124,ITR_CAT_135,ITR_CAT_56,ITR_CAT_113,ITR_CAT_126,ITR_CAT_116,ITR_CAT_84,ITR_CAT_51,ITR_CAT_117,ITR_CAT_105,ITR_CAT_81,ITR_CAT_54,ITR_CAT_101,ITR_CAT_134,ITR_CAT_71,ITR_CAT_62,ITR_CAT_78,ITR_CAT_138,ITR_CAT_17,ITR_CAT_41,ITR_CAT_69,ADJ_TYP_BALANCE_TRANSFER,ADJ_TYP_CREDIT_UNRETURNED_EQUI,ADJ_TYP_TELEVISION_PRODUCT,ADJ_TYP_OTHERS,ADJ_TYP_COLLECTION_CHARGES,ADJ_TYP_GOODWILL_CREDIT,ADJ_TYP_EQUIPMENT,ADJ_TYP_INTERNET_USAGE,ADJ_TYP_INSTALLATION_FEES,ADJ_TYP_PHONE_PRODUCT,ADJ_TYP_TELEPHONY_USAGE,ADJ_TYP_EARLY_TERMINATION_FEES,ADJ_TYP_VIDEO_ON_DEMAND,ADJ_TYP_OTHER_PAYMENTS,ADJ_TYP_RECOVERY_OF_CREDIT_WRI,ADJ_TYP_LATE_FEES,ADJ_TYP_ADMIN_FEES,ADJ_TYP_OTHER_FEES,ADJ_TYP_INTERNET_PRODUCT,ADJ_TYP_ACTIVATION_FEES,ADJ_TYP_REFUSED_BY_PAYOR_OTHER,ADJ_TYP_RATE_CHANGE,ADJ_TYP_CREDIT_BALANCE,ADJ_TYP_PAYMENT_DEBIT_AUTHORIZ,ADJ_TYP_PAY_PER_VIEW,ADJ_TYP_RECONNECT_FEES,CUSTOMER_ID,PAYMENT_METHOD,AGE,LANGUAGE,GENDER,REGION
0,31,0,0,6,103870,150900,150900,150900,150900,148360,0,0,142638.333333,103870.0,150900.0,1.452778,6,161185.000000,170520,117380,117380,170520,170520,170520,170520,167650,1.452718,0,0.000000,0,0,0,0,0,0,0.0,0.0,0.0,1,0,1,0,0,0,0,2,0,2,0,0,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4,0,0,0,0,0,0,3,0,0,0,1,30,0,5,0,1,0,0,-6355.714286,158527.619048,-7181.428571,179138.571429,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,385655,Other,27.0,first_language,UNKNOWN,0
1,429,362,0,6,123090,152930,152930,152930,152930,152930,0,0,147956.666667,123090.0,152930.0,1.242424,6,172810.000000,172810,172810,172810,172810,172810,172810,345620,0,1.000000,0,0.000000,0,0,0,0,0,0,0.0,0.0,0.0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,2,0,0,0,0,0,0,0,0,0,0,2,8,10,3,0,0,1,0,-4262.857143,158613.809524,-34562.000000,276496.000000,0.0,0.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,446885,Other,65.0,first_language,UNKNOWN,0
2,97,102,0,6,121680,116130,116280,115980,120330,120180,0,0,118430.000000,115980.0,121680.0,1.049146,6,133831.666667,137500,131060,137500,131230,131400,131060,135980,135820,1.049138,3,950.000000,2850,0,0,0,0,0,2850.0,2850.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,0,0,0,0,1,0,0,0,1,13,5,0,0,1,0,0,

# Слияние датафреймов

In [4]:
train_df_merge = train_df.merge(customer_products, on='CUSTOMER_ID', how='left')
train_df_merge

,TENURE,CITY,is_churn,BILLS_NUM,INV_MON_1,INV_MON_2,INV_MON_3,INV_MON_4,INV_MON_5,INV_MON_6,N_LATE_FEES,N_NON_SETTLED,AVG_BILL,MIN_BILL,MAX_BILL,MAX2MIN_BILL,PAY_NUM,AVG_PAY,MAX_PAY,MIN_PAY,PAY_MON_1,PAY_MON_2,PAY_MON_3,PAY_MON_4,PAY_MON_5,PAY_MON_6,MAX2MIN_PAY,ADJ_NUM,AVG_ADJ,ADJ_MON_1,ADJ_MON_2,ADJ_MON_3,ADJ_MON_4,ADJ_MON_5,ADJ_MON_6,MIN_ADJ,MAX_ADJ,MAXMINDIFF_ADJ,DSON_NUM_BUNDLE,DSON_MON_1_BUNDLE,DSON_MON_2_BUNDLE,DSON_MON_3_BUNDLE,DSON_MON_4_BUNDLE,DSON_MON_5_BUNDLE,DSON_MON_6_BUNDLE,DSON_NUM_PROMO,DSON_MON_1_PROMO,DSON_MON_2_PROMO,DSON_MON_3_PROMO,DSON_MON_4_PROMO,DSON_MON_5_PROMO,DSON_MON_6_PROMO,DSOF_NUM_BUNDLE,DSOF_MON_1_BUNDLE,DSOF_MON_2_BUNDLE,DSOF_MON_3_BUNDLE,DSOF_MON_4_BUNDLE,DSOF_MON_5_BUNDLE,DSOF_MON_6_BUNDLE,DSOF_NUM_PROMO,DSOF_MON_1_PROMO,DSOF_MON_2_PROMO,DSOF_MON_3_PROMO,DSOF_MON_4_PROMO,DSOF_MON_5_PROMO,DSOF_MON_6_PROMO,INTERACTS_NUM,TFT,SUPPORT,SOCIAL_MEDIA,OUTBOUND_CALL,MANAGE_ACCT,INQUIRY,INBOUND_CALL,ESCALATION,EMAIL,CALL_BACK,CTI,PLAY_TV,PLAY_PHONE,PLAY_INET,SINGLE_PLAY,DOUBLE_PLAY,TRIPLE_PLAY,UNKNOWN_PLAY,INV_MON_A,INV_MON_B,PAY_MON_A,PAY_MON_B,ADJ_MON_A,ADJ_MON_B,DSON_MONPROMO_A,DSON_MONPROMO_B,DSOF_MONPROMO_A,DSOF_MONPROMO_B,DSON_MONBUNDLE_A,DSON_MONBUNDLE_B,DSOF_MONBUNDLE_A,DSOF_MONBUNDLE_B,ITR_CAT_119,ITR_CAT_120,ITR_CAT_114,ITR_CAT_9,ITR_CAT_123,ITR_CAT_128,ITR_CAT_102,ITR_CAT_104,ITR_CAT_115,ITR_CAT_8,ITR_CAT_122,ITR_CAT_0,ITR_CAT_22,ITR_CAT_133,ITR_CAT_124,ITR_CAT_135,ITR_CAT_56,ITR_CAT_113,ITR_CAT_126,ITR_CAT_116,ITR_CAT_84,ITR_CAT_51,ITR_CAT_117,ITR_CAT_105,ITR_CAT_81,ITR_CAT_54,ITR_CAT_101,ITR_CAT_134,ITR_CAT_71,ITR_CAT_62,ITR_CAT_78,ITR_CAT_138,ITR_CAT_17,ITR_CAT_41,ITR_CAT_69,ADJ_TYP_BALANCE_TRANSFER,ADJ_TYP_CREDIT_UNRETURNED_EQUI,ADJ_TYP_TELEVISION_PRODUCT,ADJ_TYP_OTHERS,ADJ_TYP_COLLECTION_CHARGES,ADJ_TYP_GOODWILL_CREDIT,ADJ_TYP_EQUIPMENT,ADJ_TYP_INTERNET_USAGE,ADJ_TYP_INSTALLATION_FEES,ADJ_TYP_PHONE_PRODUCT,ADJ_TYP_TELEPHONY_USAGE,ADJ_TYP_EARLY_TERMINATION_FEES,ADJ_TYP_VIDEO_ON_DEMAND,ADJ_TYP_OTHER_PAYMENTS,ADJ_TYP_RECOVERY_OF_CREDIT_WRI,ADJ_TYP_LATE_FEES,ADJ_TYP_ADMIN_FEES,ADJ_TYP_OTHER_FEES,ADJ_TYP_INTERNET_PRODUCT,ADJ_TYP_ACTIVATION_FEES,ADJ_TYP_REFUSED_BY_PAYOR_OTHER,ADJ_TYP_RATE_CHANGE,ADJ_TYP_CREDIT_BALANCE,ADJ_TYP_PAYMENT_DEBIT_AUTHORIZ,ADJ_TYP_PAY_PER_VIEW,ADJ_TYP_RECONNECT_FEES,CUSTOMER_ID,PAYMENT_METHOD,AGE,LANGUAGE,GENDER,REGION,PRODUCT_NAME=0,PRODUCT_NAME=1,PRODUCT_NAME=2,PRODUCT_NAME=3,PRODUCT_NAME=4,PRODUCT_NAME=5,PRODUCT_NAME=6,PRODUCT_NAME=7,PRODUCT_NAME=8,PRODUCT_NAME=9,PRODUCT_NAME=10,PRODUCT_NAME=11,PRODUCT_NAME=12,PRODUCT_NAME=13,PRODUCT_NAME=14,PRODUCT_NAME=15,PRODUCT_NAME=16,PRODUCT_NAME=17,PRODUCT_NAME=18,PRODUCT_NAME=19,PRODUCT_NAME=20,PRODUCT_NAME=21,PRODUCT_NAME=22,PRODUCT_NAME=23,PRODUCT_NAME=24,PRODUCT_NAME=25,PRODUCT_NAME=26,PRODUCT_NAME=27,PRODUCT_NAME=28,PRODUCT_NAME=29,PRODUCT_NAME=30,PRODUCT_NAME=31,PRODUCT_NAME=32,PRODUCT_NAME=33,PRODUCT_NAME=34,PRODUCT_NAME=35,PRODUCT_NAME=36,PRODUCT_NAME=37,PRODUCT_NAME=38,PRODUCT_NAME=39,PRODUCT_NAME=40,PRODUCT_NAME=41,PRODUCT_NAME=42,PRODUCT_NAME=43,PRODUCT_NAME=44,PRODUCT_NAME=45,PRODUCT_NAME=46,PRODUCT_NAME=47,PRODUCT_NAME=48,PRODUCT_NAME=49,PRODUCT_NAME=50,PRODUCT_NAME=51,PRODUCT_NAME=52,PRODUCT_NAME=53,PRODUCT_NAME=54,PRODUCT_NAME=55,PRODUCT_NAME=56,PRODUCT_NAME=57,PRODUCT_NAME=58,PRODUCT_NAME=59,PRODUCT_NAME=60,PRODUCT_NAME=61,PRODUCT_NAME=62,PRODUCT_NAME=63,PRODUCT_NAME=64,PRODUCT_NAME=65,PRODUCT_NAME=66,PRODUCT_NAME=67,PRODUCT_NAME=68,PRODUCT_NAME=69,PRODUCT_NAME=70,PRODUCT_NAME=71,PRODUCT_NAME=72,PRODUCT_NAME=73,PRODUCT_NAME=74,PRODUCT_NAME=75,PRODUCT_NAME=76,PRODUCT_NAME=77,PRODUCT_NAME=78,PRODUCT_NAME=79,PRODUCT_NAME=80,PRODUCT_NAME=81,PRODUCT_NAME=82,PRODUCT_NAME=83,PRODUCT_NAME=84,PRODUCT_NAME=85,PRODUCT_NAME=86,PRODUCT_NAME=87,PRODUCT_NAME=88,PRODUCT_NAME=89,PRODUCT_NAME=90,PRODUCT_NAME=91,PRODUCT_NAME=92,PRODUCT_NAME=93,PRODUCT_NAME=94,PRODUCT_NAME=95,PRODUCT_NAME=96,PRODUCT_NAME=97,PRODUCT_NAME=98,PRODUCT_NAME=99,PRODUCT_NAME=100,PRODUCT_NAME=101,PRODUCT_NAME=102,PRODUCT_NAME=103,PRODUCT_NAME=104,PRODUC

In [5]:
ohe_columns = ['CITY', 'TFT', 'SUPPORT', 'SOCIAL_MEDIA', 'OUTBOUND_CALL', 'MANAGE_ACCT', 'INQUIRY', 'INBOUND_CALL', 'ESCALATION', 'EMAIL', 'CALL_BACK', 'CTI', 'PLAY_TV', 'PLAY_PHONE', 'PLAY_INET', 'SINGLE_PLAY', 'DOUBLE_PLAY', 'TRIPLE_PLAY', 'UNKNOWN_PLAY', 'ITR_CAT_119', 'ITR_CAT_120', 'ITR_CAT_114', 'ITR_CAT_9', 'ITR_CAT_123', 'ITR_CAT_128', 'ITR_CAT_102', 'ITR_CAT_104', 'ITR_CAT_115', 'ITR_CAT_8', 'ITR_CAT_122', 'ITR_CAT_0', 'ITR_CAT_22', 'ITR_CAT_133', 'ITR_CAT_124', 'ITR_CAT_135', 'ITR_CAT_56', 'ITR_CAT_113', 'ITR_CAT_126', 'ITR_CAT_116', 'ITR_CAT_84', 'ITR_CAT_51', 'ITR_CAT_117', 'ITR_CAT_105', 'ITR_CAT_81', 'ITR_CAT_54', 'ITR_CAT_101', 'ITR_CAT_134', 'ITR_CAT_71', 'ITR_CAT_62', 'ITR_CAT_78', 'ITR_CAT_138', 'ITR_CAT_17', 'ITR_CAT_41', 'ITR_CAT_69', 'ADJ_TYP_BALANCE_TRANSFER', 'ADJ_TYP_CREDIT_UNRETURNED_EQUI', 'ADJ_TYP_TELEVISION_PRODUCT', 'ADJ_TYP_OTHERS', 'ADJ_TYP_COLLECTION_CHARGES', 'ADJ_TYP_GOODWILL_CREDIT', 'ADJ_TYP_EQUIPMENT', 'ADJ_TYP_INTERNET_USAGE', 'ADJ_TYP_INSTALLATION_FEES', 'ADJ_TYP_PHONE_PRODUCT', 'ADJ_TYP_TELEPHONY_USAGE', 'ADJ_TYP_EARLY_TERMINATION_FEES', 'ADJ_TYP_VIDEO_ON_DEMAND', 'ADJ_TYP_OTHER_PAYMENTS', 'ADJ_TYP_RECOVERY_OF_CREDIT_WRI', 'ADJ_TYP_LATE_FEES', 'ADJ_TYP_ADMIN_FEES', 'ADJ_TYP_OTHER_FEES', 'ADJ_TYP_INTERNET_PRODUCT', 'ADJ_TYP_ACTIVATION_FEES', 'ADJ_TYP_REFUSED_BY_PAYOR_OTHER', 'ADJ_TYP_RATE_CHANGE', 'ADJ_TYP_CREDIT_BALANCE', 'ADJ_TYP_PAYMENT_DEBIT_AUTHORIZ', 'ADJ_TYP_PAY_PER_VIEW', 'ADJ_TYP_RECONNECT_FEES', 'PAYMENT_METHOD', 'LANGUAGE', 'GENDER', 'REGION']
train_df_merge["AGE"] = train_df_merge["AGE"].fillna(train_df_merge["AGE"].mean())
train_df_merge = train_df_merge.replace(np.nan, '', regex=True)
train_df_merge[train_df_merge[ohe_columns].select_dtypes(include=float).columns] = train_df_merge[train_df_merge[ohe_columns].select_dtypes(include=float).columns].astype(int)
train_df_merge.drop('CUSTOMER_ID', axis=1, inplace=True)
# train_churn = train_df_merge['is_churn'].copy()
# train_df_merge.drop('is_churn', axis=1, inplace=True)
train_df_merge

,TENURE,CITY,is_churn,BILLS_NUM,INV_MON_1,INV_MON_2,INV_MON_3,INV_MON_4,INV_MON_5,INV_MON_6,N_LATE_FEES,N_NON_SETTLED,AVG_BILL,MIN_BILL,MAX_BILL,MAX2MIN_BILL,PAY_NUM,AVG_PAY,MAX_PAY,MIN_PAY,PAY_MON_1,PAY_MON_2,PAY_MON_3,PAY_MON_4,PAY_MON_5,PAY_MON_6,MAX2MIN_PAY,ADJ_NUM,AVG_ADJ,ADJ_MON_1,ADJ_MON_2,ADJ_MON_3,ADJ_MON_4,ADJ_MON_5,ADJ_MON_6,MIN_ADJ,MAX_ADJ,MAXMINDIFF_ADJ,DSON_NUM_BUNDLE,DSON_MON_1_BUNDLE,DSON_MON_2_BUNDLE,DSON_MON_3_BUNDLE,DSON_MON_4_BUNDLE,DSON_MON_5_BUNDLE,DSON_MON_6_BUNDLE,DSON_NUM_PROMO,DSON_MON_1_PROMO,DSON_MON_2_PROMO,DSON_MON_3_PROMO,DSON_MON_4_PROMO,DSON_MON_5_PROMO,DSON_MON_6_PROMO,DSOF_NUM_BUNDLE,DSOF_MON_1_BUNDLE,DSOF_MON_2_BUNDLE,DSOF_MON_3_BUNDLE,DSOF_MON_4_BUNDLE,DSOF_MON_5_BUNDLE,DSOF_MON_6_BUNDLE,DSOF_NUM_PROMO,DSOF_MON_1_PROMO,DSOF_MON_2_PROMO,DSOF_MON_3_PROMO,DSOF_MON_4_PROMO,DSOF_MON_5_PROMO,DSOF_MON_6_PROMO,INTERACTS_NUM,TFT,SUPPORT,SOCIAL_MEDIA,OUTBOUND_CALL,MANAGE_ACCT,INQUIRY,INBOUND_CALL,ESCALATION,EMAIL,CALL_BACK,CTI,PLAY_TV,PLAY_PHONE,PLAY_INET,SINGLE_PLAY,DOUBLE_PLAY,TRIPLE_PLAY,UNKNOWN_PLAY,INV_MON_A,INV_MON_B,PAY_MON_A,PAY_MON_B,ADJ_MON_A,ADJ_MON_B,DSON_MONPROMO_A,DSON_MONPROMO_B,DSOF_MONPROMO_A,DSOF_MONPROMO_B,DSON_MONBUNDLE_A,DSON_MONBUNDLE_B,DSOF_MONBUNDLE_A,DSOF_MONBUNDLE_B,ITR_CAT_119,ITR_CAT_120,ITR_CAT_114,ITR_CAT_9,ITR_CAT_123,ITR_CAT_128,ITR_CAT_102,ITR_CAT_104,ITR_CAT_115,ITR_CAT_8,ITR_CAT_122,ITR_CAT_0,ITR_CAT_22,ITR_CAT_133,ITR_CAT_124,ITR_CAT_135,ITR_CAT_56,ITR_CAT_113,ITR_CAT_126,ITR_CAT_116,ITR_CAT_84,ITR_CAT_51,ITR_CAT_117,ITR_CAT_105,ITR_CAT_81,ITR_CAT_54,ITR_CAT_101,ITR_CAT_134,ITR_CAT_71,ITR_CAT_62,ITR_CAT_78,ITR_CAT_138,ITR_CAT_17,ITR_CAT_41,ITR_CAT_69,ADJ_TYP_BALANCE_TRANSFER,ADJ_TYP_CREDIT_UNRETURNED_EQUI,ADJ_TYP_TELEVISION_PRODUCT,ADJ_TYP_OTHERS,ADJ_TYP_COLLECTION_CHARGES,ADJ_TYP_GOODWILL_CREDIT,ADJ_TYP_EQUIPMENT,ADJ_TYP_INTERNET_USAGE,ADJ_TYP_INSTALLATION_FEES,ADJ_TYP_PHONE_PRODUCT,ADJ_TYP_TELEPHONY_USAGE,ADJ_TYP_EARLY_TERMINATION_FEES,ADJ_TYP_VIDEO_ON_DEMAND,ADJ_TYP_OTHER_PAYMENTS,ADJ_TYP_RECOVERY_OF_CREDIT_WRI,ADJ_TYP_LATE_FEES,ADJ_TYP_ADMIN_FEES,ADJ_TYP_OTHER_FEES,ADJ_TYP_INTERNET_PRODUCT,ADJ_TYP_ACTIVATION_FEES,ADJ_TYP_REFUSED_BY_PAYOR_OTHER,ADJ_TYP_RATE_CHANGE,ADJ_TYP_CREDIT_BALANCE,ADJ_TYP_PAYMENT_DEBIT_AUTHORIZ,ADJ_TYP_PAY_PER_VIEW,ADJ_TYP_RECONNECT_FEES,PAYMENT_METHOD,AGE,LANGUAGE,GENDER,REGION,PRODUCT_NAME=0,PRODUCT_NAME=1,PRODUCT_NAME=2,PRODUCT_NAME=3,PRODUCT_NAME=4,PRODUCT_NAME=5,PRODUCT_NAME=6,PRODUCT_NAME=7,PRODUCT_NAME=8,PRODUCT_NAME=9,PRODUCT_NAME=10,PRODUCT_NAME=11,PRODUCT_NAME=12,PRODUCT_NAME=13,PRODUCT_NAME=14,PRODUCT_NAME=15,PRODUCT_NAME=16,PRODUCT_NAME=17,PRODUCT_NAME=18,PRODUCT_NAME=19,PRODUCT_NAME=20,PRODUCT_NAME=21,PRODUCT_NAME=22,PRODUCT_NAME=23,PRODUCT_NAME=24,PRODUCT_NAME=25,PRODUCT_NAME=26,PRODUCT_NAME=27,PRODUCT_NAME=28,PRODUCT_NAME=29,PRODUCT_NAME=30,PRODUCT_NAME=31,PRODUCT_NAME=32,PRODUCT_NAME=33,PRODUCT_NAME=34,PRODUCT_NAME=35,PRODUCT_NAME=36,PRODUCT_NAME=37,PRODUCT_NAME=38,PRODUCT_NAME=39,PRODUCT_NAME=40,PRODUCT_NAME=41,PRODUCT_NAME=42,PRODUCT_NAME=43,PRODUCT_NAME=44,PRODUCT_NAME=45,PRODUCT_NAME=46,PRODUCT_NAME=47,PRODUCT_NAME=48,PRODUCT_NAME=49,PRODUCT_NAME=50,PRODUCT_NAME=51,PRODUCT_NAME=52,PRODUCT_NAME=53,PRODUCT_NAME=54,PRODUCT_NAME=55,PRODUCT_NAME=56,PRODUCT_NAME=57,PRODUCT_NAME=58,PRODUCT_NAME=59,PRODUCT_NAME=60,PRODUCT_NAME=61,PRODUCT_NAME=62,PRODUCT_NAME=63,PRODUCT_NAME=64,PRODUCT_NAME=65,PRODUCT_NAME=66,PRODUCT_NAME=67,PRODUCT_NAME=68,PRODUCT_NAME=69,PRODUCT_NAME=70,PRODUCT_NAME=71,PRODUCT_NAME=72,PRODUCT_NAME=73,PRODUCT_NAME=74,PRODUCT_NAME=75,PRODUCT_NAME=76,PRODUCT_NAME=77,PRODUCT_NAME=78,PRODUCT_NAME=79,PRODUCT_NAME=80,PRODUCT_NAME=81,PRODUCT_NAME=82,PRODUCT_NAME=83,PRODUCT_NAME=84,PRODUCT_NAME=85,PRODUCT_NAME=86,PRODUCT_NAME=87,PRODUCT_NAME=88,PRODUCT_NAME=89,PRODUCT_NAME=90,PRODUCT_NAME=91,PRODUCT_NAME=92,PRODUCT_NAME=93,PRODUCT_NAME=94,PRODUCT_NAME=95,PRODUCT_NAME=96,PRODUCT_NAME=97,PRODUCT_NAME=98,PRODUCT_NAME=99,PRODUCT_NAME=100,PRODUCT_NAME=101,PRODUCT_NAME=102,PRODUCT_NAME=103,PRODUCT_NAME=104,PRODUCT_NAME=105,P

In [6]:
train_df_merge.to_csv('train_preprocessed.csv', index=False)

In [7]:
test_df = pd.read_csv('test.csv')
test_df

,TENURE,CITY,BILLS_NUM,INV_MON_1,INV_MON_2,INV_MON_3,INV_MON_4,INV_MON_5,INV_MON_6,N_LATE_FEES,N_NON_SETTLED,AVG_BILL,MIN_BILL,MAX_BILL,MAX2MIN_BILL,PAY_NUM,AVG_PAY,MAX_PAY,MIN_PAY,PAY_MON_1,PAY_MON_2,PAY_MON_3,PAY_MON_4,PAY_MON_5,PAY_MON_6,MAX2MIN_PAY,ADJ_NUM,AVG_ADJ,ADJ_MON_1,ADJ_MON_2,ADJ_MON_3,ADJ_MON_4,ADJ_MON_5,ADJ_MON_6,MIN_ADJ,MAX_ADJ,MAXMINDIFF_ADJ,DSON_NUM_BUNDLE,DSON_MON_1_BUNDLE,DSON_MON_2_BUNDLE,DSON_MON_3_BUNDLE,DSON_MON_4_BUNDLE,DSON_MON_5_BUNDLE,DSON_MON_6_BUNDLE,DSON_NUM_PROMO,DSON_MON_1_PROMO,DSON_MON_2_PROMO,DSON_MON_3_PROMO,DSON_MON_4_PROMO,DSON_MON_5_PROMO,DSON_MON_6_PROMO,DSOF_NUM_BUNDLE,DSOF_MON_1_BUNDLE,DSOF_MON_2_BUNDLE,DSOF_MON_3_BUNDLE,DSOF_MON_4_BUNDLE,DSOF_MON_5_BUNDLE,DSOF_MON_6_BUNDLE,DSOF_NUM_PROMO,DSOF_MON_1_PROMO,DSOF_MON_2_PROMO,DSOF_MON_3_PROMO,DSOF_MON_4_PROMO,DSOF_MON_5_PROMO,DSOF_MON_6_PROMO,INTERACTS_NUM,TFT,SUPPORT,SOCIAL_MEDIA,OUTBOUND_CALL,MANAGE_ACCT,INQUIRY,INBOUND_CALL,ESCALATION,EMAIL,CALL_BACK,CTI,PLAY_TV,PLAY_PHONE,PLAY_INET,SINGLE_PLAY,DOUBLE_PLAY,TRIPLE_PLAY,UNKNOWN_PLAY,INV_MON_A,INV_MON_B,PAY_MON_A,PAY_MON_B,ADJ_MON_A,ADJ_MON_B,DSON_MONPROMO_A,DSON_MONPROMO_B,DSOF_MONPROMO_A,DSOF_MONPROMO_B,DSON_MONBUNDLE_A,DSON_MONBUNDLE_B,DSOF_MONBUNDLE_A,DSOF_MONBUNDLE_B,ITR_CAT_119,ITR_CAT_120,ITR_CAT_114,ITR_CAT_9,ITR_CAT_123,ITR_CAT_128,ITR_CAT_102,ITR_CAT_104,ITR_CAT_115,ITR_CAT_8,ITR_CAT_122,ITR_CAT_0,ITR_CAT_22,ITR_CAT_133,ITR_CAT_124,ITR_CAT_135,ITR_CAT_56,ITR_CAT_113,ITR_CAT_126,ITR_CAT_116,ITR_CAT_84,ITR_CAT_51,ITR_CAT_117,ITR_CAT_105,ITR_CAT_81,ITR_CAT_54,ITR_CAT_101,ITR_CAT_134,ITR_CAT_71,ITR_CAT_62,ITR_CAT_78,ITR_CAT_138,ITR_CAT_17,ITR_CAT_41,ITR_CAT_69,ADJ_TYP_BALANCE_TRANSFER,ADJ_TYP_CREDIT_UNRETURNED_EQUI,ADJ_TYP_TELEVISION_PRODUCT,ADJ_TYP_OTHERS,ADJ_TYP_COLLECTION_CHARGES,ADJ_TYP_GOODWILL_CREDIT,ADJ_TYP_EQUIPMENT,ADJ_TYP_INTERNET_USAGE,ADJ_TYP_INSTALLATION_FEES,ADJ_TYP_PHONE_PRODUCT,ADJ_TYP_TELEPHONY_USAGE,ADJ_TYP_EARLY_TERMINATION_FEES,ADJ_TYP_VIDEO_ON_DEMAND,ADJ_TYP_OTHER_PAYMENTS,ADJ_TYP_RECOVERY_OF_CREDIT_WRI,ADJ_TYP_LATE_FEES,ADJ_TYP_ADMIN_FEES,ADJ_TYP_OTHER_FEES,ADJ_TYP_INTERNET_PRODUCT,ADJ_TYP_ACTIVATION_FEES,ADJ_TYP_REFUSED_BY_PAYOR_OTHER,ADJ_TYP_RATE_CHANGE,ADJ_TYP_CREDIT_BALANCE,ADJ_TYP_PAYMENT_DEBIT_AUTHORIZ,ADJ_TYP_PAY_PER_VIEW,ADJ_TYP_RECONNECT_FEES,CUSTOMER_ID,PAYMENT_METHOD,AGE,LANGUAGE,GENDER,REGION
0,220,25,6,80000,73160,80990,80990,80990,80990,0,0,79520.00000,73160,80990,1.107026,6,9.033667e+04,91520,87810,87810,91520,91520,91520,91520,88130,1.042250,1,35000.0,0,35000,0,0,0,0,35000,35000,0,1,0,1,0,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,1,0,7,0,0,0,3,9,0,4,0,1,0,0,-812.571429,81551.42857,-45.714286,9.045095e+04,0.0,35000.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0,5,2,0,0,2,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,445171,Other,80.0,first_language,UNKNOWN,0
1,379,239,6,240880,160880,160880,160880,160880,139620,0,0,170670.00000,139620,240880,1.725254,6,1.928517e+05,272190,157760,272190,181790,181790,181790,181790,157760,1.725342,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,4,0,0,0,1,9,5,4,0,0,1,0,14465.714290,134505.71430,16347.142860,1.519838e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,652721,Pre-authorized_Debit,70.0,first_language,FEMALE,0
2,120,41,6,93940,107840,86990,107840,93940,80010,0,0,95093.33333,80010,107840,1.347832,6,1.074517e+05,121850,90410,228000,98300,0,121850,106150,90410,1.347749,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,24,0,0,1,0,0,0,2585.714286,88629.04762,26733.000000,7.547600e+04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,288131,Other,52.0,first_language,UNKNOWN,0
3,6,362,6,85990,85990,85990,18200,146450,83990,0,0,8

In [8]:
test_df_merge = test_df.merge(customer_products, on='CUSTOMER_ID', how='left')
test_df_merge

,TENURE,CITY,BILLS_NUM,INV_MON_1,INV_MON_2,INV_MON_3,INV_MON_4,INV_MON_5,INV_MON_6,N_LATE_FEES,N_NON_SETTLED,AVG_BILL,MIN_BILL,MAX_BILL,MAX2MIN_BILL,PAY_NUM,AVG_PAY,MAX_PAY,MIN_PAY,PAY_MON_1,PAY_MON_2,PAY_MON_3,PAY_MON_4,PAY_MON_5,PAY_MON_6,MAX2MIN_PAY,ADJ_NUM,AVG_ADJ,ADJ_MON_1,ADJ_MON_2,ADJ_MON_3,ADJ_MON_4,ADJ_MON_5,ADJ_MON_6,MIN_ADJ,MAX_ADJ,MAXMINDIFF_ADJ,DSON_NUM_BUNDLE,DSON_MON_1_BUNDLE,DSON_MON_2_BUNDLE,DSON_MON_3_BUNDLE,DSON_MON_4_BUNDLE,DSON_MON_5_BUNDLE,DSON_MON_6_BUNDLE,DSON_NUM_PROMO,DSON_MON_1_PROMO,DSON_MON_2_PROMO,DSON_MON_3_PROMO,DSON_MON_4_PROMO,DSON_MON_5_PROMO,DSON_MON_6_PROMO,DSOF_NUM_BUNDLE,DSOF_MON_1_BUNDLE,DSOF_MON_2_BUNDLE,DSOF_MON_3_BUNDLE,DSOF_MON_4_BUNDLE,DSOF_MON_5_BUNDLE,DSOF_MON_6_BUNDLE,DSOF_NUM_PROMO,DSOF_MON_1_PROMO,DSOF_MON_2_PROMO,DSOF_MON_3_PROMO,DSOF_MON_4_PROMO,DSOF_MON_5_PROMO,DSOF_MON_6_PROMO,INTERACTS_NUM,TFT,SUPPORT,SOCIAL_MEDIA,OUTBOUND_CALL,MANAGE_ACCT,INQUIRY,INBOUND_CALL,ESCALATION,EMAIL,CALL_BACK,CTI,PLAY_TV,PLAY_PHONE,PLAY_INET,SINGLE_PLAY,DOUBLE_PLAY,TRIPLE_PLAY,UNKNOWN_PLAY,INV_MON_A,INV_MON_B,PAY_MON_A,PAY_MON_B,ADJ_MON_A,ADJ_MON_B,DSON_MONPROMO_A,DSON_MONPROMO_B,DSOF_MONPROMO_A,DSOF_MONPROMO_B,DSON_MONBUNDLE_A,DSON_MONBUNDLE_B,DSOF_MONBUNDLE_A,DSOF_MONBUNDLE_B,ITR_CAT_119,ITR_CAT_120,ITR_CAT_114,ITR_CAT_9,ITR_CAT_123,ITR_CAT_128,ITR_CAT_102,ITR_CAT_104,ITR_CAT_115,ITR_CAT_8,ITR_CAT_122,ITR_CAT_0,ITR_CAT_22,ITR_CAT_133,ITR_CAT_124,ITR_CAT_135,ITR_CAT_56,ITR_CAT_113,ITR_CAT_126,ITR_CAT_116,ITR_CAT_84,ITR_CAT_51,ITR_CAT_117,ITR_CAT_105,ITR_CAT_81,ITR_CAT_54,ITR_CAT_101,ITR_CAT_134,ITR_CAT_71,ITR_CAT_62,ITR_CAT_78,ITR_CAT_138,ITR_CAT_17,ITR_CAT_41,ITR_CAT_69,ADJ_TYP_BALANCE_TRANSFER,ADJ_TYP_CREDIT_UNRETURNED_EQUI,ADJ_TYP_TELEVISION_PRODUCT,ADJ_TYP_OTHERS,ADJ_TYP_COLLECTION_CHARGES,ADJ_TYP_GOODWILL_CREDIT,ADJ_TYP_EQUIPMENT,ADJ_TYP_INTERNET_USAGE,ADJ_TYP_INSTALLATION_FEES,ADJ_TYP_PHONE_PRODUCT,ADJ_TYP_TELEPHONY_USAGE,ADJ_TYP_EARLY_TERMINATION_FEES,ADJ_TYP_VIDEO_ON_DEMAND,ADJ_TYP_OTHER_PAYMENTS,ADJ_TYP_RECOVERY_OF_CREDIT_WRI,ADJ_TYP_LATE_FEES,ADJ_TYP_ADMIN_FEES,ADJ_TYP_OTHER_FEES,ADJ_TYP_INTERNET_PRODUCT,ADJ_TYP_ACTIVATION_FEES,ADJ_TYP_REFUSED_BY_PAYOR_OTHER,ADJ_TYP_RATE_CHANGE,ADJ_TYP_CREDIT_BALANCE,ADJ_TYP_PAYMENT_DEBIT_AUTHORIZ,ADJ_TYP_PAY_PER_VIEW,ADJ_TYP_RECONNECT_FEES,CUSTOMER_ID,PAYMENT_METHOD,AGE,LANGUAGE,GENDER,REGION,PRODUCT_NAME=0,PRODUCT_NAME=1,PRODUCT_NAME=2,PRODUCT_NAME=3,PRODUCT_NAME=4,PRODUCT_NAME=5,PRODUCT_NAME=6,PRODUCT_NAME=7,PRODUCT_NAME=8,PRODUCT_NAME=9,PRODUCT_NAME=10,PRODUCT_NAME=11,PRODUCT_NAME=12,PRODUCT_NAME=13,PRODUCT_NAME=14,PRODUCT_NAME=15,PRODUCT_NAME=16,PRODUCT_NAME=17,PRODUCT_NAME=18,PRODUCT_NAME=19,PRODUCT_NAME=20,PRODUCT_NAME=21,PRODUCT_NAME=22,PRODUCT_NAME=23,PRODUCT_NAME=24,PRODUCT_NAME=25,PRODUCT_NAME=26,PRODUCT_NAME=27,PRODUCT_NAME=28,PRODUCT_NAME=29,PRODUCT_NAME=30,PRODUCT_NAME=31,PRODUCT_NAME=32,PRODUCT_NAME=33,PRODUCT_NAME=34,PRODUCT_NAME=35,PRODUCT_NAME=36,PRODUCT_NAME=37,PRODUCT_NAME=38,PRODUCT_NAME=39,PRODUCT_NAME=40,PRODUCT_NAME=41,PRODUCT_NAME=42,PRODUCT_NAME=43,PRODUCT_NAME=44,PRODUCT_NAME=45,PRODUCT_NAME=46,PRODUCT_NAME=47,PRODUCT_NAME=48,PRODUCT_NAME=49,PRODUCT_NAME=50,PRODUCT_NAME=51,PRODUCT_NAME=52,PRODUCT_NAME=53,PRODUCT_NAME=54,PRODUCT_NAME=55,PRODUCT_NAME=56,PRODUCT_NAME=57,PRODUCT_NAME=58,PRODUCT_NAME=59,PRODUCT_NAME=60,PRODUCT_NAME=61,PRODUCT_NAME=62,PRODUCT_NAME=63,PRODUCT_NAME=64,PRODUCT_NAME=65,PRODUCT_NAME=66,PRODUCT_NAME=67,PRODUCT_NAME=68,PRODUCT_NAME=69,PRODUCT_NAME=70,PRODUCT_NAME=71,PRODUCT_NAME=72,PRODUCT_NAME=73,PRODUCT_NAME=74,PRODUCT_NAME=75,PRODUCT_NAME=76,PRODUCT_NAME=77,PRODUCT_NAME=78,PRODUCT_NAME=79,PRODUCT_NAME=80,PRODUCT_NAME=81,PRODUCT_NAME=82,PRODUCT_NAME=83,PRODUCT_NAME=84,PRODUCT_NAME=85,PRODUCT_NAME=86,PRODUCT_NAME=87,PRODUCT_NAME=88,PRODUCT_NAME=89,PRODUCT_NAME=90,PRODUCT_NAME=91,PRODUCT_NAME=92,PRODUCT_NAME=93,PRODUCT_NAME=94,PRODUCT_NAME=95,PRODUCT_NAME=96,PRODUCT_NAME=97,PRODUCT_NAME=98,PRODUCT_NAME=99,PRODUCT_NAME=100,PRODUCT_NAME=101,PRODUCT_NAME=102,PRODUCT_NAME=103,PRODUCT_NAME=104,PRODUCT_NAME=10

In [9]:
test_df_merge["AGE"] = test_df_merge["AGE"].fillna(test_df_merge["AGE"].mean())
test_df_merge = test_df_merge.replace(np.nan, '', regex=True)
test_df_merge[test_df_merge[ohe_columns].select_dtypes(include=float).columns] = test_df_merge[test_df_merge[ohe_columns].select_dtypes(include=float).columns].astype(int)
test_df_merge.drop('CUSTOMER_ID', axis=1, inplace=True)
test_df_merge

,TENURE,CITY,BILLS_NUM,INV_MON_1,INV_MON_2,INV_MON_3,INV_MON_4,INV_MON_5,INV_MON_6,N_LATE_FEES,N_NON_SETTLED,AVG_BILL,MIN_BILL,MAX_BILL,MAX2MIN_BILL,PAY_NUM,AVG_PAY,MAX_PAY,MIN_PAY,PAY_MON_1,PAY_MON_2,PAY_MON_3,PAY_MON_4,PAY_MON_5,PAY_MON_6,MAX2MIN_PAY,ADJ_NUM,AVG_ADJ,ADJ_MON_1,ADJ_MON_2,ADJ_MON_3,ADJ_MON_4,ADJ_MON_5,ADJ_MON_6,MIN_ADJ,MAX_ADJ,MAXMINDIFF_ADJ,DSON_NUM_BUNDLE,DSON_MON_1_BUNDLE,DSON_MON_2_BUNDLE,DSON_MON_3_BUNDLE,DSON_MON_4_BUNDLE,DSON_MON_5_BUNDLE,DSON_MON_6_BUNDLE,DSON_NUM_PROMO,DSON_MON_1_PROMO,DSON_MON_2_PROMO,DSON_MON_3_PROMO,DSON_MON_4_PROMO,DSON_MON_5_PROMO,DSON_MON_6_PROMO,DSOF_NUM_BUNDLE,DSOF_MON_1_BUNDLE,DSOF_MON_2_BUNDLE,DSOF_MON_3_BUNDLE,DSOF_MON_4_BUNDLE,DSOF_MON_5_BUNDLE,DSOF_MON_6_BUNDLE,DSOF_NUM_PROMO,DSOF_MON_1_PROMO,DSOF_MON_2_PROMO,DSOF_MON_3_PROMO,DSOF_MON_4_PROMO,DSOF_MON_5_PROMO,DSOF_MON_6_PROMO,INTERACTS_NUM,TFT,SUPPORT,SOCIAL_MEDIA,OUTBOUND_CALL,MANAGE_ACCT,INQUIRY,INBOUND_CALL,ESCALATION,EMAIL,CALL_BACK,CTI,PLAY_TV,PLAY_PHONE,PLAY_INET,SINGLE_PLAY,DOUBLE_PLAY,TRIPLE_PLAY,UNKNOWN_PLAY,INV_MON_A,INV_MON_B,PAY_MON_A,PAY_MON_B,ADJ_MON_A,ADJ_MON_B,DSON_MONPROMO_A,DSON_MONPROMO_B,DSOF_MONPROMO_A,DSOF_MONPROMO_B,DSON_MONBUNDLE_A,DSON_MONBUNDLE_B,DSOF_MONBUNDLE_A,DSOF_MONBUNDLE_B,ITR_CAT_119,ITR_CAT_120,ITR_CAT_114,ITR_CAT_9,ITR_CAT_123,ITR_CAT_128,ITR_CAT_102,ITR_CAT_104,ITR_CAT_115,ITR_CAT_8,ITR_CAT_122,ITR_CAT_0,ITR_CAT_22,ITR_CAT_133,ITR_CAT_124,ITR_CAT_135,ITR_CAT_56,ITR_CAT_113,ITR_CAT_126,ITR_CAT_116,ITR_CAT_84,ITR_CAT_51,ITR_CAT_117,ITR_CAT_105,ITR_CAT_81,ITR_CAT_54,ITR_CAT_101,ITR_CAT_134,ITR_CAT_71,ITR_CAT_62,ITR_CAT_78,ITR_CAT_138,ITR_CAT_17,ITR_CAT_41,ITR_CAT_69,ADJ_TYP_BALANCE_TRANSFER,ADJ_TYP_CREDIT_UNRETURNED_EQUI,ADJ_TYP_TELEVISION_PRODUCT,ADJ_TYP_OTHERS,ADJ_TYP_COLLECTION_CHARGES,ADJ_TYP_GOODWILL_CREDIT,ADJ_TYP_EQUIPMENT,ADJ_TYP_INTERNET_USAGE,ADJ_TYP_INSTALLATION_FEES,ADJ_TYP_PHONE_PRODUCT,ADJ_TYP_TELEPHONY_USAGE,ADJ_TYP_EARLY_TERMINATION_FEES,ADJ_TYP_VIDEO_ON_DEMAND,ADJ_TYP_OTHER_PAYMENTS,ADJ_TYP_RECOVERY_OF_CREDIT_WRI,ADJ_TYP_LATE_FEES,ADJ_TYP_ADMIN_FEES,ADJ_TYP_OTHER_FEES,ADJ_TYP_INTERNET_PRODUCT,ADJ_TYP_ACTIVATION_FEES,ADJ_TYP_REFUSED_BY_PAYOR_OTHER,ADJ_TYP_RATE_CHANGE,ADJ_TYP_CREDIT_BALANCE,ADJ_TYP_PAYMENT_DEBIT_AUTHORIZ,ADJ_TYP_PAY_PER_VIEW,ADJ_TYP_RECONNECT_FEES,PAYMENT_METHOD,AGE,LANGUAGE,GENDER,REGION,PRODUCT_NAME=0,PRODUCT_NAME=1,PRODUCT_NAME=2,PRODUCT_NAME=3,PRODUCT_NAME=4,PRODUCT_NAME=5,PRODUCT_NAME=6,PRODUCT_NAME=7,PRODUCT_NAME=8,PRODUCT_NAME=9,PRODUCT_NAME=10,PRODUCT_NAME=11,PRODUCT_NAME=12,PRODUCT_NAME=13,PRODUCT_NAME=14,PRODUCT_NAME=15,PRODUCT_NAME=16,PRODUCT_NAME=17,PRODUCT_NAME=18,PRODUCT_NAME=19,PRODUCT_NAME=20,PRODUCT_NAME=21,PRODUCT_NAME=22,PRODUCT_NAME=23,PRODUCT_NAME=24,PRODUCT_NAME=25,PRODUCT_NAME=26,PRODUCT_NAME=27,PRODUCT_NAME=28,PRODUCT_NAME=29,PRODUCT_NAME=30,PRODUCT_NAME=31,PRODUCT_NAME=32,PRODUCT_NAME=33,PRODUCT_NAME=34,PRODUCT_NAME=35,PRODUCT_NAME=36,PRODUCT_NAME=37,PRODUCT_NAME=38,PRODUCT_NAME=39,PRODUCT_NAME=40,PRODUCT_NAME=41,PRODUCT_NAME=42,PRODUCT_NAME=43,PRODUCT_NAME=44,PRODUCT_NAME=45,PRODUCT_NAME=46,PRODUCT_NAME=47,PRODUCT_NAME=48,PRODUCT_NAME=49,PRODUCT_NAME=50,PRODUCT_NAME=51,PRODUCT_NAME=52,PRODUCT_NAME=53,PRODUCT_NAME=54,PRODUCT_NAME=55,PRODUCT_NAME=56,PRODUCT_NAME=57,PRODUCT_NAME=58,PRODUCT_NAME=59,PRODUCT_NAME=60,PRODUCT_NAME=61,PRODUCT_NAME=62,PRODUCT_NAME=63,PRODUCT_NAME=64,PRODUCT_NAME=65,PRODUCT_NAME=66,PRODUCT_NAME=67,PRODUCT_NAME=68,PRODUCT_NAME=69,PRODUCT_NAME=70,PRODUCT_NAME=71,PRODUCT_NAME=72,PRODUCT_NAME=73,PRODUCT_NAME=74,PRODUCT_NAME=75,PRODUCT_NAME=76,PRODUCT_NAME=77,PRODUCT_NAME=78,PRODUCT_NAME=79,PRODUCT_NAME=80,PRODUCT_NAME=81,PRODUCT_NAME=82,PRODUCT_NAME=83,PRODUCT_NAME=84,PRODUCT_NAME=85,PRODUCT_NAME=86,PRODUCT_NAME=87,PRODUCT_NAME=88,PRODUCT_NAME=89,PRODUCT_NAME=90,PRODUCT_NAME=91,PRODUCT_NAME=92,PRODUCT_NAME=93,PRODUCT_NAME=94,PRODUCT_NAME=95,PRODUCT_NAME=96,PRODUCT_NAME=97,PRODUCT_NAME=98,PRODUCT_NAME=99,PRODUCT_NAME=100,PRODUCT_NAME=101,PRODUCT_NAME=102,PRODUCT_NAME=103,PRODUCT_NAME=104,PRODUCT_NAME=105,PRODUCT_NA

In [10]:
test_df_merge.to_csv('test_preprocessed.csv', index=False)